In [181]:
%load_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import numpy as np
from IPython.core.debugger import set_trace
from tqdm.notebook import trange, tqdm
import json
import os
import re
import datetime
from time import sleep
import pytz
KST = pytz.timezone('Asia/Seoul')

import snscrape
import snscrape.modules.twitter as sntwitter
import gspread
import pydata_google_auth
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 빅쿼리 접속
### (참고) pandas-gbq를 쓰지 않는다. pandas-gbq는 struct, repeated 자료형의 입력을 지원하지 않기 때문에

In [182]:
credentials = pydata_google_auth.get_user_credentials(
  ['https://www.googleapis.com/auth/bigquery'],
)

In [183]:
bq = bigquery.Client(project='sideb-proejct', credentials=credentials)
dataset = bq.get_dataset('moiber_20210920'); dataset

Dataset(DatasetReference('sideb-proejct', 'moiber_20210920'))

In [184]:
twitter_feed = bq.get_table('sideb-proejct.moiber_20210920.twitter_feed')
twitter_user = bq.get_table('sideb-proejct.moiber_20210920.twitter_user')

In [185]:
twitter_feed.schema

[SchemaField('id', 'INTEGER', 'REQUIRED', None, (), ()),
 SchemaField('url', 'STRING', 'REQUIRED', None, (), ()),
 SchemaField('created', 'TIMESTAMP', 'REQUIRED', None, (), ()),
 SchemaField('content', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('userid', 'INTEGER', 'REQUIRED', None, (), ()),
 SchemaField('replyCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('retweetCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('likeCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('quoteCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('lang', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('sourceLabel', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('media', 'STRING', 'REPEATED', None, (), ()),
 SchemaField('inReplyToUserid', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('mentionedUserids', 'INTEGER', 'REPEATED', None, (), ()),
 SchemaField('loaded', 'TIMESTAMP', 'NULLABLE', None, (), ())]

In [186]:
twitter_user.schema

[SchemaField('id', 'INTEGER', 'REQUIRED', None, (), ()),
 SchemaField('username', 'STRING', 'REQUIRED', None, (), ()),
 SchemaField('displayname', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('description', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('verified', 'BOOLEAN', 'NULLABLE', None, (), ()),
 SchemaField('created', 'TIMESTAMP', 'NULLABLE', None, (), ()),
 SchemaField('followersCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('friendsCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('statusesCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('favouritesCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('listedCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('mediaCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('profileImageUrl', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('loaded', 'TIMESTAMP', 'NULLABLE', None, (), ())]

# 테스트 쿼리

In [138]:
n_max = 10
since = '2021-03-01'
until = '2021-08-31'
baseqry = '#032c OR #032씨'
qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry, since=since, until=until); qry

'#032c OR #032씨 since:2021-03-01 until:2021-08-31 lang:ko'

In [139]:
tweets_list = []

for i, _tw in enumerate(sntwitter.TwitterSearchScraper(qry).get_items()):
    if i > n_max:
        break
        
    tweets_list.append(_tw)
    
tweets_list

[Tweet(url='https://twitter.com/ateezstyles/status/1394850512365887491', date=datetime.datetime(2021, 5, 19, 3, 0, 51, tzinfo=datetime.timezone.utc), content='[210518] #HONGJOONG • 홍중 모자 티셔츠 정보\n➖\n#032C Pyrate Society Cap\n#SAINTLAURENT Striped Monogram T-Shirt in Wool Jersey\n#whathongjoongwear \n➖\n#ATEEZ #에이티즈 #김홍중 #홍중 @ATEEZofficial https://t.co/1x5HXx5vCo', renderedContent='[210518] #HONGJOONG • 홍중 모자 티셔츠 정보\n➖\n#032C Pyrate Society Cap\n#SAINTLAURENT Striped Monogram T-Shirt in Wool Jersey\n#whathongjoongwear \n➖\n#ATEEZ #에이티즈 #김홍중 #홍중 @ATEEZofficial https://t.co/1x5HXx5vCo', id=1394850512365887491, user=User(username='ateezstyles', id=1285019843800915968, displayname='ATEEZ STYLES 에이티즈 옷장', description='(very slow) #에이티즈 의 옷 정보 계정입니다. 제보/요청은 DM 💌', rawDescription='(very slow) #에이티즈 의 옷 정보 계정입니다. 제보/요청은 DM 💌', descriptionUrls=None, verified=False, created=datetime.datetime(2020, 7, 20, 1, 13, 12, tzinfo=datetime.timezone.utc), followersCount=6063, friendsCount=5, statusesCount=1

In [165]:
feed = tweets_list[0]; feed.__dict__

{'url': 'https://twitter.com/ateezstyles/status/1394850512365887491',
 'date': datetime.datetime(2021, 5, 19, 3, 0, 51, tzinfo=datetime.timezone.utc),
 'content': '[210518] #HONGJOONG • 홍중 모자 티셔츠 정보\n➖\n#032C Pyrate Society Cap\n#SAINTLAURENT Striped Monogram T-Shirt in Wool Jersey\n#whathongjoongwear \n➖\n#ATEEZ #에이티즈 #김홍중 #홍중 @ATEEZofficial https://t.co/1x5HXx5vCo',
 'renderedContent': '[210518] #HONGJOONG • 홍중 모자 티셔츠 정보\n➖\n#032C Pyrate Society Cap\n#SAINTLAURENT Striped Monogram T-Shirt in Wool Jersey\n#whathongjoongwear \n➖\n#ATEEZ #에이티즈 #김홍중 #홍중 @ATEEZofficial https://t.co/1x5HXx5vCo',
 'id': 1394850512365887491,
 'user': User(username='ateezstyles', id=1285019843800915968, displayname='ATEEZ STYLES 에이티즈 옷장', description='(very slow) #에이티즈 의 옷 정보 계정입니다. 제보/요청은 DM 💌', rawDescription='(very slow) #에이티즈 의 옷 정보 계정입니다. 제보/요청은 DM 💌', descriptionUrls=None, verified=False, created=datetime.datetime(2020, 7, 20, 1, 13, 12, tzinfo=datetime.timezone.utc), followersCount=6063, friendsCount=5

In [161]:
media = feed.media[0]

def get_url(m):
    try:
        return m.fullUrl
    except:
        return m.thumbnailUrl
    
get_url(media)

'https://pbs.twimg.com/ext_tw_video_thumb/1384774873071820800/pu/img/_FloyymZ9cnWc7eb.jpg'

In [133]:
tloaded = datetime.datetime.now(datetime.timezone.utc)

users = []
user = feed.user
in_reply_to_user = feed.inReplyToUser
mentioned_users = feed.mentionedUsers

if mentioned_users:
    users += mentioned_users

if user and (user not in users):
    users += [user]
    
if in_reply_to_user and (in_reply_to_user not in users):
    users += [in_reply_to_user]

In [134]:
users

[User(username='YouTube', id=10228272, displayname='YouTube', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friendsCount=None, statusesCount=None, favouritesCount=None, listedCount=None, mediaCount=None, location=None, protected=None, linkUrl=None, linkTcourl=None, profileImageUrl=None, profileBannerUrl=None),
 User(username='RieL_Cho', id=183958456, displayname='RieL', description='fun^10 x int^40 = Ir2', rawDescription='fun^10 x int^40 = Ir2', descriptionUrls=None, verified=False, created=datetime.datetime(2010, 8, 28, 9, 24, 7, tzinfo=datetime.timezone.utc), followersCount=267, friendsCount=338, statusesCount=30149, favouritesCount=11480, listedCount=4, mediaCount=7425, location='👁\u200d🗨', protected=False, linkUrl=None, linkTcourl=None, profileImageUrl='https://pbs.twimg.com/profile_images/1440658908989689865/6SD9dhnS_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/183958456/1631635310')]

In [157]:
_twitter_feeds = [(
    feed.id,
    feed.url,
    feed.date,
    feed.content,
    user.id,
    feed.replyCount,
    feed.retweetCount,
    feed.likeCount,
    feed.quoteCount,
    feed.lang,
    feed.sourceLabel,
    [get_url(_d) for _d in feed.media] if feed.media else None,
    in_reply_to_user.id if in_reply_to_user else None,
    [_d.id for _d in mentioned_users] if mentioned_users else None,
    tloaded
)]

_twitter_feeds

[(1384778936589111296,
  'https://twitter.com/Originals_kr/status/1384778936589111296',
  datetime.datetime(2021, 4, 21, 8, 0, 1, tzinfo=datetime.timezone.utc),
  'adidas by 032C 4월 22일 발매 예정 \n\n우리 주위를 둘러싼 환경에서 영감을 받아 새롭게 탄생한 \n두 가지의 디자인 032C GSG TR와 032C GSG Mule을 선보입니다. \n\n[판매처]\n*아디다스 공식 온라인 스토어\n\n#아디다스오리지널스 #adidasOriginals #032C https://t.co/NpXjm6ltt7',
  183958456,
  0,
  0,
  2,
  0,
  'ko',
  'TweetDeck',
  ['https://pbs.twimg.com/ext_tw_video_thumb/1384774873071820800/pu/img/_FloyymZ9cnWc7eb.jpg'],
  None,
  [10228272],
  datetime.datetime(2021, 9, 23, 13, 28, 19, 850159, tzinfo=datetime.timezone.utc))]

In [106]:
_twitter_users = [(
    u.id,
    u.username,
    u.displayname,
    u.description,
    u.verified,
    u.created,
    u.followersCount,
    u.friendsCount,
    u.statusesCount,
    u.favouritesCount,
    u.listedCount,
    u.mediaCount,
    u.profileImageUrl,
    tloaded
) for u in users]

_twitter_users

[(1118400718547763200,
  'kashi_cats',
  '카시 in manyang',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  datetime.datetime(2021, 9, 23, 12, 52, 50, 633197, tzinfo=datetime.timezone.utc)),
 (1414146942574071812,
  'hjwldsaccuse',
  '고소',
  '괴물 주원동식 여진구 신하균 | 주식 한정 리버스❌',
  False,
  datetime.datetime(2021, 7, 11, 8, 58, 6, tzinfo=datetime.timezone.utc),
  243,
  186,
  6478,
  14978,
  2,
  365,
  'https://pbs.twimg.com/profile_images/1431330602146402305/-bLPLiPw_normal.jpg',
  datetime.datetime(2021, 9, 23, 12, 52, 50, 633197, tzinfo=datetime.timezone.utc))]

In [107]:
bq.insert_rows(twitter_feed, _twitter_feeds)

[]

In [108]:
bq.insert_rows(twitter_user, _twitter_users)

[]

In [175]:
def get_url(m):
    try:
        return m.fullUrl
    except:
        return m.thumbnailUrl
    

def populate(feed):
    tloaded = datetime.datetime.now(datetime.timezone.utc)

    users = []
    user = feed.user
    in_reply_to_user = feed.inReplyToUser
    mentioned_users = feed.mentionedUsers

    if mentioned_users:
        users += mentioned_users

    if user and (user not in users):
        users += [user]

    if in_reply_to_user and (in_reply_to_user not in users):
        users += [in_reply_to_user]
        
    _twitter_feeds = [(
        feed.id,
        feed.url,
        feed.date,
        feed.content,
        user.id,
        feed.replyCount,
        feed.retweetCount,
        feed.likeCount,
        feed.quoteCount,
        feed.lang,
        feed.sourceLabel,
        [get_url(_d) for _d in feed.media] if feed.media else None,
        in_reply_to_user.id if in_reply_to_user else None,
        [_d.id for _d in mentioned_users] if mentioned_users else None,
        tloaded
    )]
    
    _twitter_users = [(
        u.id,
        u.username,
        u.displayname,
        u.description,
        u.verified,
        u.created,
        u.followersCount,
        u.friendsCount,
        u.statusesCount,
        u.favouritesCount,
        u.listedCount,
        u.mediaCount,
        u.profileImageUrl,
        tloaded
    ) for u in users]
    
    bq.insert_rows(twitter_feed, _twitter_feeds)
    bq.insert_rows(twitter_user, _twitter_users)

In [110]:
populate(feed)

# 스프레드시트에서 트위터 쿼리정보 가져오기

In [187]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]

json_file_name = 'sideb-proejct-066e081b7bcd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1roV5_RPFtJ6z40I6EYGRmqx01Fe6JOOmaJ6nbDrfmkw/edit#gid=2020655845'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet = doc.worksheet('20210913')
values = worksheet.get_all_values()
header, rows = values[0], values[1:]

df = pd.DataFrame(rows, columns=header)
df = df[['구분','한글명','영문명 full','영문명 short','트위터 검색쿼리','검색잘됨']].set_index('영문명 short')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [198]:
qrys = df[df['검색잘됨']=='1']['트위터 검색쿼리'][146:268]; qrys

영문명 short
moscot                   moscot OR 모스코트 OR "모스콧" -"존 모스콧" -baseball -선발...
moschino                                      "모스키노" OR "모스치노" OR moschino
montbell                                montbell OR #몽벨 -from:montbell_xxx
montblanc                                  #montblanc 몽블랑 -"final fantasy"
monclear                  monclear OR "몽클레르" OR "몽클레어" OR "몽끌레르" OR "몽끌레어"
                                               ...                        
advisoryboardcrystals    advisoryboardcrystals OR "어드바이저리보드크리스털" OR #어드...
urbanstoff                                           urbanstoff OR "어반스터프"
a6ove                                                      "a6ove" OR #어보브
awakeny                                                awakeny OR "어웨이크뉴욕"
acoldwall                                      acoldwall OR #acw OR "어콜드월"
Name: 트위터 검색쿼리, Length: 122, dtype: object

# 크롤링 실행

In [ ]:
since = '2021-03-01'
until = '2021-08-31'

for brand, baseqry in tqdm(list(qrys.items())[:]):
    qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry.strip(), since=since, until=until)
    
    for i, _feed in enumerate(sntwitter.TwitterSearchScraper(qry).get_items()):
        populate(_feed)
    
    print(brand + ' ' + str(i+1))
    sleep(10)

  0%|          | 0/122 [00:00<?, ?it/s]

moscot 55
moschino 1299
montbell 4
montblanc 18
monclear 1569
muji 5902
midnightstudios 5902
mischief 240
miumiu 2123
mizuno 28
millet 1
baracuta 31
barbour 1
vans 10214
balenciaga 5377
valentino 123
bally 11
balmain 624
barrel 6


In [179]:
'test' + str(5)

'test5'